In [1]:
from keras.models import Model, load_model
from keras.layers import Input, Conv2D, AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D, BatchNormalization
from keras.layers import ReLU, Add, Dense, Flatten, Concatenate, ZeroPadding2D, SeparableConv2D
from keras.losses import BinaryCrossentropy, CategoricalCrossentropy
#from keras.layers import RandomFlip, RandomRotation, RandomZoom, AutoContrast
from keras_cv.layers import RandomFlip, RandomRotation, RandomZoom, AutoContrast
from keras.optimizers import Adam, RMSprop
from keras.utils import load_img, img_to_array, to_categorical, plot_model
from keras.regularizers import L1L2
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.applications import Xception

import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split

In [2]:
DIR_TRAIN = "./train/"
DIR_TEST = "./test/"

PATH_TRAIN = "./train.csv"
PATH_TEST = "./test.csv"

In [3]:
data = pd.read_csv(PATH_TRAIN, delimiter=',', header=0)
data.head()

num_classes = int(data['class'].max()) + 1

In [4]:
import shutil

DIR_VAL = "./val/"

def data_for_dataset():
    
    for i in range(num_classes):
        dir = os.path.join(DIR_TRAIN, str(i))
        if not os.path.exists(dir):
            os.mkdir(dir)
        for f in data[data['class']==i].values:
            fn = os.path.join(dir, f[0])
            if not os.path.exists(fn):
                shutil.move(os.path.join(DIR_TRAIN, f[0]), fn)
    
    if not os.path.exists(DIR_VAL):
        os.mkdir(DIR_VAL)
    for i in range(num_classes):
        dir = os.path.join(DIR_VAL, str(i))
        if not os.path.exists(dir):
            os.mkdir(dir)
        ls = sorted(os.listdir(os.path.join(DIR_TRAIN, str(i))))
        for j in range(len(ls)//10):
            ffrom = os.path.join(DIR_TRAIN, str(i), ls[j])
            fto = os.path.join(DIR_VAL, str(i), ls[j])
            if os.path.exists(ffrom) and not os.path.exists(fto):
                shutil.move(ffrom, fto)

#data_for_dataset()


In [5]:
kshape = 128
shape = (kshape, kshape)
xtrain = np.array([img_to_array(load_img(os.path.join(DIR_TRAIN, s), color_mode='rgb', target_size=shape, keep_aspect_ratio=True)) for s in data.ID_img]) / 255.
ytrain = np.array([to_categorical(s, num_classes=num_classes) for s in data['class']])
#ytrain = np.array([int(s) for s in data['class']])

xtrain, xtest, ytrain, ytest = train_test_split(xtrain, ytrain, test_size=.1)
print(xtrain.shape, xtest.shape, ytrain.shape, ytest.shape)
#print(xtrain.shape, ytrain.shape)

(4491, 128, 128, 3) (499, 128, 128, 3) (4491, 8) (499, 8)


In [6]:
#from keras.preprocessing.image import ImageDataGenerator

#datagen = ImageDataGenerator(featurewise_center=True, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, validation_split=0.1)#horizontal_flip=True
#datagen.fit(xtrain)

In [7]:
def model1():
    l1l2 = L1L2(l1=1e-5, l2=1e-5)
    #, kernel_regularizer=l1l2
    
    inp = Input(shape=(kshape, kshape, 3))

    x = Conv2D(8, 3, padding='valid', kernel_regularizer=l1l2)(inp)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    for k, n in zip([16, 32, 64, 128, 256], [2,2,3,3,3]):
        for i in range(n):
            y = Conv2D(k, 3, padding='same', kernel_regularizer=l1l2)(x)
            y = BatchNormalization()(y)
            y = ReLU()(y)
            y = Conv2D(k, 3, padding='same', kernel_regularizer=l1l2)(y)
            
            x = Conv2D(k, 1, padding='same', kernel_regularizer=l1l2)(x)
            
            if i == n-1:
                y = MaxPooling2D(2)(y)
                x = MaxPooling2D(2)(x)
            
            x = Add()([x,y])
            x = BatchNormalization()(x)
            x = ReLU()(x)
    
    x = Conv2D(512, 3, padding='valid', kernel_regularizer=l1l2)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    #x = Flatten()(x)
    x = GlobalMaxPooling2D()(x)

    for i in range(2):
        x = Dense(512, kernel_regularizer=l1l2)(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)

    x = Dense(num_classes, activation='sigmoid', kernel_regularizer=l1l2)(x)

    model = Model(inp, x)

    return model, './ulite.h5'

In [8]:
def model2():
    l1l2 = L1L2(l1=1e-5, l2=1e-5)
    #, kernel_regularizer=l1l2
    
    inp = Input(shape=(kshape, kshape, 3))
    x = BatchNormalization()(inp)

    #x = RandomFlip('vertical')(x)
    #x = RandomRotation(0.3)(x)
    #x = RandomZoom(.2, .2)(x)
    #x = AutoContrast([0, 1])(x)
    
    for k in [64, 128, 256]:
        x = Conv2D(k, 3, strides=2, padding='valid', kernel_regularizer=l1l2)(x)
        #x = SeparableConv2D (k, 3, strides=2, padding='valid', kernel_regularizer=l1l2, use_bias=False)(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)

    a = []
    for i in range(16):
        y = Conv2D(64, 3, padding='valid', kernel_regularizer=l1l2)(x)
        #y = SeparableConv2D (64, 3, padding='valid', kernel_regularizer=l1l2, use_bias=False)(x)
        y = BatchNormalization()(y)
        y = ReLU()(y)
        y = Conv2D(64, 3, strides=2, padding='valid', kernel_regularizer=l1l2)(y)
        #y = SeparableConv2D (64, 3, strides=2, padding='valid', kernel_regularizer=l1l2, use_bias=False)(y)
        a.append(y)
    
    x = Concatenate()(a)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(512, 3, padding='valid', kernel_regularizer=l1l2)(x)
    #x = SeparableConv2D (512, 3, padding='valid', kernel_regularizer=l1l2, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    #x = Flatten()(x)
    x = GlobalAveragePooling2D()(x)

    for i in range(2):
        x = Dense(512, kernel_regularizer=l1l2)(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)

    x = Dense(num_classes, activation='softmax')(x)

    model = Model(inp, x)
    model.compile(optimizer=RMSprop(learning_rate=1e-4), loss='categorical_crossentropy', metrics='accuracy')

    return model, './ulite2.h5'

In [9]:
def model3():
    base_model = Xception(include_top=False, input_shape=(kshape, kshape, 3), pooling='avg')
    base_model.trainable = True

    inputs = Input(shape=(kshape,kshape, 3))
    x = base_model(inputs, training=True)
    
    for i in range(2):
        x = Dense(256)(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
    
    x = Dense(8, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=x)
    
    return model, './ulite3.h5'

In [34]:
model, path_weights = model2()
plot_model(model, show_layer_names=True, show_shapes=True, dpi=64)

es = EarlyStopping(patience=10, verbose=1, restore_best_weights=True)
mcp = ModelCheckpoint(path_weights, monitor='val_loss', mode='min', save_best_only=True, save_weights_only=True)
rop = ReduceLROnPlateau(patience=8, min_lr=1e-10, verbose=1)


In [42]:
model.compile(optimizer=Adam(learning_rate=5e-3), loss='categorical_crossentropy', metrics='accuracy')

for i in range(2):
    try:
        model.load_weights(path_weights)
    except:
        print('mlyaa')

    #history = model.fit(datagen.flow(xtrain, ytrain, batch_size=100, subset='training'), epochs=100, verbose=1, validation_data=datagen.flow(xtrain, ytrain, batch_size=100, subset='validation'), callbacks=[es, mcp, rop])
    history = model.fit(xtrain, ytrain, batch_size=100, epochs=500, verbose=1, validation_data=(xtest, ytest), callbacks=[es, mcp, rop])

Epoch 1/500
45/45 [==============================] - 10s 161ms/step - loss: 0.3708 - accuracy: 0.9757 - val_loss: 2.1025 - val_accuracy: 0.7174 - lr: 0.0050
Epoch 2/500
45/45 [==============================] - 6s 144ms/step - loss: 0.3848 - accuracy: 0.9695 - val_loss: 2.0106 - val_accuracy: 0.7776 - lr: 0.0050
Epoch 3/500
45/45 [==============================] - 6s 144ms/step - loss: 0.3081 - accuracy: 0.9855 - val_loss: 0.7997 - val_accuracy: 0.8597 - lr: 0.0050
Epoch 4/500
45/45 [==============================] - 6s 144ms/step - loss: 0.2732 - accuracy: 0.9922 - val_loss: 0.7198 - val_accuracy: 0.8677 - lr: 0.0050
Epoch 5/500
45/45 [==============================] - 6s 143ms/step - loss: 0.2874 - accuracy: 0.9822 - val_loss: 1.2313 - val_accuracy: 0.8277 - lr: 0.0050
Epoch 6/500
45/45 [==============================] - 6s 143ms/step - loss: 0.3121 - accuracy: 0.9806 - val_loss: 1.1637 - val_accuracy: 0.8437 - lr: 0.0050
Epoch 7/500
45/45 [==============================] - 6s 143ms/s

In [49]:
model.load_weights(path_weights)
model.evaluate(xtest, ytest)

16/16 [==============================] - 0s 19ms/step - loss: 0.3342 - accuracy: 0.9679


[0.33417776226997375, 0.9679358601570129]

In [50]:
data = pd.read_csv(PATH_TEST, delimiter=',', header=0)

In [51]:
xval = np.array([img_to_array(load_img(os.path.join(DIR_TEST, s), color_mode='rgb', target_size=shape, keep_aspect_ratio=True)) for s in data.ID_img]) / 255.

In [52]:
pred = model.predict(xval)
pred = [np.argmax(i) for i in pred]

67/67 [==============================] - 1s 15ms/step


In [53]:
del(data['class'])
data['class'] = pred
data.head(10)

,ID_img,class
0,0.jpg,5
1,1.jpg,5
2,2.jpg,1
3,3.jpg,1
4,4.jpg,6
5,5.jpg,3
6,6.jpg,7
7,7.jpg,3
8,8.jpg,3
9,9.jpg,5


In [54]:
data.to_csv('./submit.csv', index=False)